### * RNN 주요 레이어 종류
#### (1) SimpleRNN :가장 간단한 형태의 RNN레이어, 활성화 함수로 tanh가 사용됨(tanh: -1 ~ 1 사이의 값을 반환)
#### (2) LSTM(Long short Term Memory) : 입력 데이터와 출력 사이의 거리가 멀어질수로 연관 관계가 적어진다(Long Term Dependency,장기의존성 문제), LSTM은 장기 의존성 문제를 해결하기 위해 출력값외에 셀상태(cell state)값을 출력함, 활성화 함수로 tanh외에 sigmoid가 사용됨
#### (3) GRU(Gated Recurent Unit) : 뉴욕대 조경현 교수 등이 제안, LSTM보다 구조가 간단하고 성능이 우수함

<<참고 : https://colah.github.io/posts/2015-08-Understanding-LSTMs/>>
https://link.springer.com/article/10.1007/s11063-009-9096-2

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
# Sequence data
X = np.array([[0,1,2,3],
              [1,2,3,4],
              [2,3,4,5],
              [3,4,5,6],
              [4,5,6,7],
              [5,6,7,8]],dtype=np.float32)

x_data = tf.reshape(X,(-1,4,1))  # (6,4,1)

y_data = np.array([4,5,6,7,8,9],dtype=np.float32)

print(x_data.shape,y_data.shape)
# print(type(x_data),type(y_data))
x_data

(6, 4, 1) (6,)


<tf.Tensor: shape=(6, 4, 1), dtype=float32, numpy=
array([[[0.],
        [1.],
        [2.],
        [3.]],

       [[1.],
        [2.],
        [3.],
        [4.]],

       [[2.],
        [3.],
        [4.],
        [5.]],

       [[3.],
        [4.],
        [5.],
        [6.]],

       [[4.],
        [5.],
        [6.],
        [7.]],

       [[5.],
        [6.],
        [7.],
        [8.]]], dtype=float32)>

### [1] SimpleRNN
#### 가장 간단한 형태의 RNN 

In [3]:
#Q1)  SimpleRNN모델 구현하자.  
#X :  (6,4,1) 
# Wx : (1,300 )   , : 입력가중치(입력차원 , 뉴런) =파라미터 개수 = 1*300 = 300
# Wh (300,300) , : 숨겨진 상태 가중치 : (300,300) = (유닛 개수,숨겨진 상태) , 파라미터 = 300 * 300 = 90,000
# b (300) , param 90600       = wx+ wh + b  = 90,600
#, OUt (None, 4, 300)  
model = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(units=300,return_sequences=True,input_shape=[4,1]),
    
#X :  (None, 4, 300)   
# Wx : (300,300 )   , 300 * 300 = 90000
#Wh (300,300) ,       300 * 300 - 90000
#    b (300) ,        90000 + 90000 + 300 = 180300    
#    param 180300   
#    , OUt (None,300)  
    tf.keras.layers.SimpleRNN(units=300),
    tf.keras.layers.Dense(1)     #출력차원이 1, 가중치 형태 = (300,1) = 300 + 1 = 301
])

model.compile(optimizer='adam',loss='mse')
model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 4, 300)            90600     
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 300)               180300    
                                                                 
 dense (Dense)               (None, 1)                 301       
                                                                 
Total params: 271201 (1.03 MB)
Trainable params: 271201 (1.03 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [4]:
model.fit(x_data,y_data,epochs=100,verbose=1)
print(model.predict(x_data))

Epoch 1/100

1/1 [==============================] - 4s 4s/step - loss: 35.6214
Epoch 2/100
1/1 [==============================] - 0s 7ms/step - loss: 0.5345
Epoch 3/100
1/1 [==============================] - 0s 10ms/step - loss: 6.2700
Epoch 4/100
1/1 [==============================] - 0s 12ms/step - loss: 6.1489
Epoch 5/100
1/1 [==============================] - 0s 8ms/step - loss: 2.4105
Epoch 6/100
1/1 [==============================] - 0s 9ms/step - loss: 1.2029
Epoch 7/100
1/1 [==============================] - 0s 7ms/step - loss: 2.9213
Epoch 8/100
1/1 [==============================] - 0s 7ms/step - loss: 3.2904
Epoch 9/100
1/1 [==============================] - 0s 7ms/step - loss: 1.9717
Epoch 10/100
1/1 [==============================] - 0s 12ms/step - loss: 0.8597
Epoch 11/100
1/1 [==============================] - 0s 8ms/step - loss: 0.7301
Epoch 12/100
1/1 [==============================] - 0s 8ms/step - loss: 1.0862
Epoch 13/100
1/1 [==============================] - 0s 8m

1/1 [==============================] - 1s 895ms/step
[[4.0117373]
 [4.957607 ]
 [5.956311 ]
 [7.057538 ]
 [8.069312 ]
 [8.893594 ]]


In [5]:
print(model.predict(np.array([[[6.],[7.],[8.],[9.]]])))
print(model.predict(np.array([[[-1.],[0.],[1.],[2.]]])))

1/1 [==============================] - 0s 64ms/step
[[9.513967]]
1/1 [==============================] - 0s 32ms/step
[[1.1378653]]


In [6]:
model.evaluate(x_data,y_data)

1/1 [==============================] - 1s 575ms/step - loss: 0.0039


0.0038801219779998064

### [2] LSTM(Long short Term Memory)
#### 입력 데이터와 출력 사이의 거리가 멀어질수로 연관 관계가 적어진다(Long Term Dependency,장기의존성 문제)
#### LSTM은 장기 의존성 문제를 해결하기 위해 출력값외에 셀상태(cell state)값을 출력함

In [7]:
#Q2) RNN 순환 신경망 구현  : LSTM
model = tf.keras.Sequential([   
 #X :  (6,4,1) 
# Wx : (1,4*300 )   , Wh (300,4*300) , b (4*300) , param  4*90600      , OUt (None, 4, 300)
    tf.keras.layers.LSTM(units=300,return_sequences=True,input_shape=[4,1]),  
    
  #X :  None, 4, 300
# Wx : (300,4*300 )   , Wh (300,4*300) , b (4*300) , param  4*180300     , OUt   (None, 300) 
    tf.keras.layers.LSTM(units=300),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam',loss='mse')
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 4, 300)            362400    
                                                                 
 lstm_1 (LSTM)               (None, 300)               721200    
                                                                 
 dense_1 (Dense)             (None, 1)                 301       
                                                                 
Total params: 1083901 (4.13 MB)
Trainable params: 1083901 (4.13 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [8]:
# 학습 및 예측
model.fit(x_data,y_data,epochs=100,verbose=1)
print(model.predict(x_data))

Epoch 1/100
1/1 [==============================] - 6s 6s/step - loss: 45.4470
Epoch 2/100
1/1 [==============================] - 0s 21ms/step - loss: 39.9041
Epoch 3/100
1/1 [==============================] - 0s 25ms/step - loss: 34.3499
Epoch 4/100
1/1 [==============================] - 0s 23ms/step - loss: 28.2107
Epoch 5/100
1/1 [==============================] - 0s 21ms/step - loss: 21.2174
Epoch 6/100
1/1 [==============================] - 0s 26ms/step - loss: 13.5783
Epoch 7/100
1/1 [==============================] - 0s 25ms/step - loss: 6.3604
Epoch 8/100
1/1 [==============================] - 0s 26ms/step - loss: 1.7415
Epoch 9/100
1/1 [==============================] - 0s 24ms/step - loss: 2.1865
Epoch 10/100
1/1 [==============================] - 0s 21ms/step - loss: 5.5998
Epoch 11/100
1/1 [==============================] - 0s 25ms/step - loss: 6.9333
Epoch 12/100
1/1 [==============================] - 0s 27ms/step - loss: 5.9104
Epoch 13/100
1/1 [===========================

In [9]:
# 학습되지 않은 입력 데이터에 대한 예측 결과
print(model.predict(np.array([[[6.],[7.],[8.],[9.]]])))
print(model.predict(np.array([[[-1.],[0.],[1.],[2.]]])))

1/1 [==============================] - 0s 28ms/step
[[8.907026]]
1/1 [==============================] - 0s 26ms/step
[[1.2317121]]


In [10]:
model.evaluate(x_data,y_data)

1/1 [==============================] - 1s 1s/step - loss: 0.1467


0.14668039977550507

### [3] GRU(Gated Recurent Unit)
#### 뉴욕대 조경현 교수 등이 제안, LSTM보다 구조가 간단하고 성능이 우수

In [11]:
# RNN 순환 신경망 구현  : GRU
model = tf.keras.Sequential([
    tf.keras.layers.GRU(units=300,return_sequences=True,input_shape=[4,1]),
    tf.keras.layers.GRU(units=300),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam',loss='mse')
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 4, 300)            272700    
                                                                 
 gru_1 (GRU)                 (None, 300)               541800    
                                                                 
 dense_2 (Dense)             (None, 1)                 301       
                                                                 
Total params: 814801 (3.11 MB)
Trainable params: 814801 (3.11 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [12]:
# 학습 및 예측
model.fit(x_data,y_data,epochs=100,verbose=1)
print(model.predict(x_data))

Epoch 1/100
1/1 [==============================] - 7s 7s/step - loss: 43.3607
Epoch 2/100
1/1 [==============================] - 0s 18ms/step - loss: 26.6226
Epoch 3/100
1/1 [==============================] - 0s 14ms/step - loss: 13.3136
Epoch 4/100
1/1 [==============================] - 0s 14ms/step - loss: 3.9350
Epoch 5/100
1/1 [==============================] - 0s 19ms/step - loss: 0.5827
Epoch 6/100
1/1 [==============================] - 0s 15ms/step - loss: 4.0200
Epoch 7/100
1/1 [==============================] - 0s 12ms/step - loss: 6.7868
Epoch 8/100
1/1 [==============================] - 0s 11ms/step - loss: 5.9054
Epoch 9/100
1/1 [==============================] - 0s 19ms/step - loss: 3.4495
Epoch 10/100
1/1 [==============================] - 0s 15ms/step - loss: 1.3455
Epoch 11/100
1/1 [==============================] - 0s 13ms/step - loss: 0.3510
Epoch 12/100
1/1 [==============================] - 0s 12ms/step - loss: 0.3494
Epoch 13/100
1/1 [==============================

In [13]:
# 학습되지 않은 입력 데이터에 대한 예측 결과
print(model.predict(np.array([[[6.],[7.],[8.],[9.]]])))
print(model.predict(np.array([[[-1.],[0.],[1.],[2.]]])))

1/1 [==============================] - 0s 27ms/step
[[9.273868]]
1/1 [==============================] - 0s 23ms/step
[[2.0855718]]
